In [15]:
import pandas as pd
import numpy as np

train_cn = pd.read_excel('./中文_trian.xlsx')
train_ja = pd.read_excel('./日语_train.xlsx')
train_en = pd.read_excel('./英文_train.xlsx')

test_ja = pd.read_excel('./testA.xlsx', sheet_name='日语_testA')
test_en = pd.read_excel('./testA.xlsx', sheet_name='英文_testA')

In [16]:
train_cn[~train_cn['槽值1'].isnull()].head(2)

,原始文本,意图,槽值1,槽值2
0,16.5度,adjust_ac_temperature_to_number,offset:16.5,NaN
1,16度,adjust_ac_temperature_to_number,offset:16,NaN


In [17]:
train_ja[~train_ja['槽值1'].isnull()].head(2)

,原始文本,中文翻译,意图,槽值1,槽值2
35,エアコンの冷房モードをオンにして,开启空调的制冷模式,open_ac_mode,mode:冷房モード,NaN
36,エアコンの冷房モードをつけて,开启空调的制冷模式,open_ac_mode,mode:冷房モード,NaN


In [18]:
train_cn['原始文本'].apply(len).max(), train_ja['原始文本'].apply(len).max(), train_en['原始文本'].apply(len).max()

(36, 33, 71)

In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# AutoTokenizer：分词器

# Auto：自动识别的
model_name = "bert-base-chinese"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [23]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig, BertModel, AutoModel
model = AutoModel.from_pretrained("hfl/chinese-roberta-wwm-ext")

Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
import torch
item = {key: torch.tensor(inputs[key]).view(1, -1) for key, val in inputs.items()}

output = model(input_ids = item['input_ids'], attention_mask = item['attention_mask'])
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.2090,  0.2921,  0.2075,  ..., -0.3150,  0.0329, -0.0352],
         [ 0.1141,  0.5552,  0.6361,  ...,  0.2087,  0.0772, -0.8664],
         [ 0.8204,  0.2228,  0.3018,  ...,  0.1404, -0.2797, -0.0781],
         ...,
         [ 0.4625,  0.1090, -0.1031,  ..., -0.0032, -0.2640, -0.5389],
         [ 0.4616,  0.1152, -0.1214,  ...,  0.0076, -0.2020, -0.6232],
         [ 0.4785,  0.1196, -0.0029,  ...,  0.0727, -0.2967, -0.5149]]],
       grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[ 0.9966,  0.9605,  0.9827,  0.9573,  0.7397,  0.2169, -0.9281, -0.2038,
          0.5496, -0.9845,  0.9994,  0.9872, -0.6430, -0.8330,  0.9501, -0.9907,
          0.6900,  0.6050,  0.2719, -0.6332,  0.9893, -0.9912, -0.8960,  0.2563,
         -0.0675,  0.7463,  0.7332,  0.4835, -0.9986,  0.9761,  0.7391,  0.9935,
          0.4609, -0.9963, -0.9994,  0.5128,  0.3549,  0.9656,  0.5940, -0.9801,
         -0.8955,  0.1684, -0.402

In [25]:
output.last_hidden_state.shape, output.pooler_output.shape

(torch.Size([1, 28, 768]), torch.Size([1, 768]))

In [26]:
for tag in ['intent', 'device', 'mode', 'offset', 'endloc', 'landmark', 'singer', 'song']:
    train_ja['槽值1'] = train_ja['槽值1'].str.replace(f'{tag}:', '')
    train_ja['槽值2'] = train_ja['槽值2'].str.replace(f'{tag}:', '')
    
    train_cn['槽值1'] = train_cn['槽值1'].str.replace(f'{tag}:', '')
    train_cn['槽值2'] = train_cn['槽值2'].str.replace(f'{tag}:', '')

    train_en['槽值1'] = train_en['槽值1'].str.replace(f'{tag}:', '')
    train_en['槽值2'] = train_en['槽值2'].str.replace(f'{tag}:', '')

In [27]:
train_df = pd.concat([
    train_ja[['原始文本', '意图', '槽值1', '槽值2']],
    train_cn[['原始文本', '意图', '槽值1', '槽值2']].sample(10000),
    train_en[['原始文本', '意图', '槽值1', '槽值2']],
],axis = 0)
train_df = train_df.sample(frac=1.0)
train_df['意图_encode'], lbl_ecode = pd.factorize(train_df['意图'])

In [28]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
config = AutoConfig.from_pretrained("bert-base-multilingual-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

In [29]:
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch
from torch import nn

# 数据集读取
class XunFeiDataset(Dataset):
    def __init__(self, encodings, intent):
        self.encodings = encodings
        self.intent = intent
    
    # 读取单个样本
    def __getitem__(self, idx):        
            
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['label'] = torch.tensor(int(self.intent[idx]))
        return item
    
    def __len__(self):
        return len(self.intent)
    
class XunFeiModel(nn.Module):
    def __init__(self, num_labels): 
        super(XunFeiModel,self).__init__() 
        self.model = model = AutoModel.from_pretrained("bert-base-multilingual-cased")
        self.dropout = nn.Dropout(0.1) 
        self.classifier = nn.Linear(768, num_labels)

    def forward(self, input_ids=None, attention_mask=None,labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = self.dropout(outputs[0]) #outputs[0]=last hidden state
        logits = self.classifier(sequence_output[:,0,:].view(-1,768))
        
        return logits

In [30]:
train_encoding = tokenizer(train_df['原始文本'].tolist()[:-500], truncation=True, padding=True, max_length=40)
val_encoding = tokenizer(train_df['原始文本'].tolist()[-500:], truncation=True, padding=True, max_length=40)

train_dataset = XunFeiDataset(train_encoding, train_df['意图_encode'].tolist()[:-500])
val_dataset = XunFeiDataset(val_encoding, train_df['意图_encode'].tolist()[-500:])

In [31]:
# 单个读取到批量读取
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
model = XunFeiModel(18)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
model = model.to(device)

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

In [ ]:
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW

loss_fn = CrossEntropyLoss() # ingore index = -1
optim = AdamW(model.parameters(), lr=5e-5)

In [ ]:
def train():
    model.train()
    total_train_loss = 0
    iter_num = 0
    total_iter = len(train_loader)
    for batch in train_loader:
        # 正向传播
        optim.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        label = batch['label'].to(device)

        pred = model(
            input_ids, 
            attention_mask
        )
        
        loss = loss_fn(pred, label)
        
        # 反向梯度信息
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # 参数更新
        optim.step()

        iter_num += 1
        
        if(iter_num % 100 == 0):
            print("iter_num: %d, loss: %.4f, %.2f%% %.4f" % (
                iter_num, loss.item(), iter_num/total_iter*100, 
                (pred.argmax(1) == label).float().data.cpu().numpy().mean(),
            ))

def validation():
    model.eval()
    label_acc = 0
    for batch in val_dataloader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            label = batch['label'].to(device)

            pred = model(
                input_ids, 
                attention_mask
            )
    
            label_acc += (pred.argmax(1) == label).float().sum().item()
    
    label_acc = label_acc / len(val_dataloader.dataset)

    print("-------------------------------")
    print("Accuracy: %.4f" % (label_acc))
    print("-------------------------------")

In [ ]:
for epoch in range(2):
    train()
    validation()

In [ ]:
def prediction():
    model.eval()
    test_label = []
    for batch in test_dataloader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            pred = model(input_ids, attention_mask)
            test_label += list(pred.argmax(1).data.cpu().numpy())
    return test_label

In [ ]:
test_encoding = tokenizer(test_en['原始文本'].tolist(), truncation=True, padding=True, max_length=40)
test_dataset = XunFeiDataset(test_encoding, [0] * len(test_en))
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

test_en_intent = prediction()

test_encoding = tokenizer(test_ja['原始文本'].tolist(), truncation=True, padding=True, max_length=40)
test_dataset = XunFeiDataset(test_encoding, [0] * len(test_ja))
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

test_ja_intent = prediction()

In [ ]:
test_ja['意图'] = [lbl_ecode[x] for x in test_ja_intent]
test_en['意图'] = [lbl_ecode[x] for x in test_en_intent]
test_en['槽值1'] = np.nan
test_en['槽值2'] = np.nan

test_ja['槽值1'] = np.nan
test_ja['槽值2'] = np.nan

writer = pd.ExcelWriter('submit.xlsx')
test_en[['意图', '槽值1', '槽值2']].to_excel(writer, sheet_name='英文_testA', index=None)
test_ja[['意图', '槽值1', '槽值2']].to_excel(writer, sheet_name='日语_testA', index=None)
writer.save()
writer.close()

In [ ]:
# 数据集读取
class XunFeiDataset2(Dataset):
    def __init__(self, encodings, label_i, label_j):
        self.encodings = encodings
        self.intent = intent
        self.tag1 = tag1
        self.tag2 = tag2
    
    # 读取单个样本
    def __getitem__(self, idx):        
        
        tags_single = [0] + list(tags_single) + [0]
        tags_single = tags_single + [0] * (32 - len(tags_single))
        tags_single = tags_single[:maxlen]
            
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['label_i'] = torch.tensor(int(self.label_i[idx]))
        item['label_j'] = torch.tensor(int(self.label_j[idx]))
        return item
    
    def __len__(self):
        return len(self.label_i)
    
class XunFeiModel2(nn.Module):
    def __init__(self, num_labels_i, num_labels_j): 
        super(XunFeiModel,self).__init__() 

        #Load Model with given checkpoint and extract its body
        self.model = AutoModel.from_pretrained("hfl/chinese-roberta-wwm-ext")
        self.dropout = nn.Dropout(0.1) 
        self.classifier_i = nn.Linear(768, num_labels_i)
        self.classifier_j = nn.Linear(768, num_labels_j)

    def forward(self, input_ids=None, attention_mask=None,labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = self.dropout(outputs[0]) #outputs[0]=last hidden state
        
        # 问诊方向
        logits_i = self.classifier_i(sequence_output[:,0,:].view(-1,768))
        # 疾病标签
        logits_j = self.classifier_j(sequence_output[:,0,:].view(-1,768))
        
        return logits_i, logits_j